In [26]:
# import required libraries
import pandas as pd
import numpy as np


In [18]:
# load the raw dataset (adjust path if needed)
df = pd.read_csv("synthetic_banking_conversations_2000.csv")

# quick peek at data and shape
print(f"Loaded shape: {df.shape}")
df.head(3)


Loaded shape: (2000, 24)


,post_id,platform,bank_name,post_text,category,subcategory,sentiment_label,sentiment_score,date,user_id,...,language,country,is_verified,topic_tags,extracted_entities,urgency_flag,complaint_flag,resolved_flag,response_time_hours,source_url
0,P000001,Reddit,First National Bank,Account flagged for suspicious activity; suppo...,Security & Fraud,Unauthorized Transaction,Positive,0.398,2024-11-23,U81482,...,en,AU,True,security,First National Bank,False,False,NaN,NaN,https://example.com/post/1
1,P000002,Yelp,First National Bank,Session keeps timing out while I complete form...,Online Banking,UX,Negative,-0.243,2023-10-12,U10851,...,en,CA,False,rewards|loan,First National Bank,True,False,NaN,NaN,https://example.com/post/2
2,P000003,Reddit,SafeTrust Bank,"Found the ATM network reliable in my city, no ...",ATM Service,Network Down,Positive,0.256,2023-02-14,U70217,...,en,AU,True,support|ux,SafeTrust Bank,False,False,NaN,NaN,https://example.com/post/3


In [19]:
# look at column names, datatypes, and missing values
print("Columns:", df.columns.tolist())
print("\nDtypes:\n", df.dtypes)
print("\nTop null counts:\n", df.isna().sum().sort_values(ascending=False).head(10))

# preview country and language distribution
print("\nCountry counts:\n", df["country"].value_counts(dropna=False).head(10))
print("\nLanguage counts:\n", df["language"].value_counts(dropna=False).head(10))


Columns: ['post_id', 'platform', 'bank_name', 'post_text', 'category', 'subcategory', 'sentiment_label', 'sentiment_score', 'date', 'user_id', 'user_followers', 'likes', 'shares', 'replies', 'language', 'country', 'is_verified', 'topic_tags', 'extracted_entities', 'urgency_flag', 'complaint_flag', 'resolved_flag', 'response_time_hours', 'source_url']

Dtypes:
 post_id                 object
platform                object
bank_name               object
post_text               object
category                object
subcategory             object
sentiment_label         object
sentiment_score        float64
date                    object
user_id                 object
user_followers           int64
likes                    int64
shares                   int64
replies                  int64
language                object
country                 object
is_verified               bool
topic_tags              object
extracted_entities      object
urgency_flag              bool
complaint_flag   

In [20]:
# normalize country formatting (strip spaces, uppercase)
df["country"] = df["country"].apply(lambda x: x.strip().upper() if isinstance(x, str) else x)

# keep only US rows
df_us = df[df["country"] == "US"].copy()

# sanity check: how many rows kept
print(f"Kept {len(df_us)} of {len(df)} rows ({len(df_us)/len(df):.1%}).")

# drop the country column now that we've filtered by it
df_us.drop(columns=["country"], inplace=True)

# quick peek after filtering
df_us.head(3)



Kept 353 of 2000 rows (17.6%).


,post_id,platform,bank_name,post_text,category,subcategory,sentiment_label,sentiment_score,date,user_id,...,replies,language,is_verified,topic_tags,extracted_entities,urgency_flag,complaint_flag,resolved_flag,response_time_hours,source_url
11,P000012,Yelp,GlobalBank,Cash shortage at ATM during the weekend ruined...,ATM Service,Cash Shortage,Negative,-1.000,2024-04-07,U25860,...,2,en,False,transfer|security|support,GlobalBank,True,False,NaN,NaN,https://example.com/post/12
14,P000015,Yelp,Maple Savings,"Spent 45 minutes on hold, then agent was unhel...",Customer Service,In-branch Help,Neutral,-0.143,2024-04-18,U42742,...,5,fr,False,fees|app|transfer,Maple Savings,False,False,NaN,NaN,https://example.com/post/15
16,P000017,Forums,Horizon Bank,Why is the monthly fee increasing? Feels like ...,Fees & Charges,Monthly Fees,Negative,-0.756,2024-07-09,U17989,...,2,en,True,transfer|support,Horizon Bank,True,False,NaN,NaN,https://example.com/post/17


In [21]:
# normalize language column (lowercase, strip spaces)
df_us["language"] = df_us["language"].astype(str).str.strip().str.lower()

# keep only rows where language == 'en'
before = len(df_us)
df_us = df_us[df_us["language"] == "en"].copy()

# sanity check: how many rows kept
print(f"Kept {len(df_us)} of {before} rows after filtering for English ({len(df_us)/before:.1%}).")

# quick peek after filtering
df_us.head(3)


Kept 291 of 353 rows after filtering for English (82.4%).


,post_id,platform,bank_name,post_text,category,subcategory,sentiment_label,sentiment_score,date,user_id,...,replies,language,is_verified,topic_tags,extracted_entities,urgency_flag,complaint_flag,resolved_flag,response_time_hours,source_url
11,P000012,Yelp,GlobalBank,Cash shortage at ATM during the weekend ruined...,ATM Service,Cash Shortage,Negative,-1.000,2024-04-07,U25860,...,2,en,False,transfer|security|support,GlobalBank,True,False,NaN,NaN,https://example.com/post/12
16,P000017,Forums,Horizon Bank,Why is the monthly fee increasing? Feels like ...,Fees & Charges,Monthly Fees,Negative,-0.756,2024-07-09,U17989,...,2,en,True,transfer|support,Horizon Bank,True,False,NaN,NaN,https://example.com/post/17
17,P000018,Reddit,Pioneer Credit,The banking app keeps crashing when I try to t...,App Performance,Login Issues,Negative,-1.039,2024-08-06,U42271,...,2,en,False,support|transfer,Pioneer Credit,False,True,NaN,NaN,https://example.com/post/18


In [22]:
# drop columns we don't need for category/sentiment analysis
cols_to_drop = [
    "topic_tags", 
    "extracted_entities", 
    "response_time_hours", 
    "urgency_flag", 
    "complaint_flag"
]

# only drop if the columns actually exist
cols_to_drop = [c for c in cols_to_drop if c in df_us.columns]
df_us.drop(columns=cols_to_drop, inplace=True)

# show remaining columns
print(f"Dropped columns: {cols_to_drop}")
print("Remaining columns:", df_us.columns.tolist())


Dropped columns: ['topic_tags', 'extracted_entities', 'response_time_hours', 'urgency_flag', 'complaint_flag']
Remaining columns: ['post_id', 'platform', 'bank_name', 'post_text', 'category', 'subcategory', 'sentiment_label', 'sentiment_score', 'date', 'user_id', 'user_followers', 'likes', 'shares', 'replies', 'language', 'is_verified', 'resolved_flag', 'source_url']


In [23]:
# standardize category values: strip spaces, collapse multi-spaces, title-case
df_us["category"] = (
    df_us["category"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
)

# preview cleaned categories
print("Sample categories:", df_us["category"].unique()[:10])
print("\nTop 10 categories by count:\n", df_us["category"].value_counts().head(10))


Sample categories: ['Atm Service' 'Fees & Charges' 'App Performance' 'Online Banking'
 'Loan Process' 'Branch Experience' 'Security & Fraud'
 'International Transfers' 'Interest Rates' 'Customer Service']

Top 10 categories by count:
 category
Atm Service          30
Branch Experience    26
Online Banking       25
Fees & Charges       24
Interest Rates       23
Customer Service     23
Account Opening      21
Credit Card          19
Loan Process         17
Notifications        17
Name: count, dtype: int64


In [24]:
# convert sentiment_score to numeric and clip to [-1, 1]
df_us["sentiment_score"] = pd.to_numeric(df_us["sentiment_score"], errors="coerce")
df_us["sentiment_score"] = df_us["sentiment_score"].clip(-1, 1)

# preview sentiment_score distribution
print(df_us["sentiment_score"].describe())
print("Sample scores:", df_us["sentiment_score"].head().tolist())



count    291.000000
mean       0.058330
std        0.613851
min       -1.000000
25%       -0.424500
50%        0.034000
75%        0.558500
max        1.000000
Name: sentiment_score, dtype: float64
Sample scores: [-1.0, -0.756, -1.0, 0.449, 0.133]


In [27]:
# normalize sentiment_label and fill missing ones using sentiment_score
def label_from_score(x):
    if pd.isna(x): return None
    if x >= 0.2: return "Positive"
    if x <= -0.2: return "Negative"
    return "Neutral"

# mark invalid labels as NaN
valid_labels = {"positive", "neutral", "negative"}
df_us["sentiment_label"] = df_us["sentiment_label"].astype(str).str.strip()
df_us.loc[~df_us["sentiment_label"].str.lower().isin(valid_labels), "sentiment_label"] = np.nan

# fill missing labels using score
df_us["sentiment_label"] = df_us["sentiment_label"].fillna(df_us["sentiment_score"].apply(label_from_score))

# capitalize for consistency
df_us["sentiment_label"] = df_us["sentiment_label"].astype(str).str.capitalize()

# preview counts
print(df_us["sentiment_label"].value_counts())
print("Sample labels:", df_us["sentiment_label"].head().tolist())



sentiment_label
Positive    126
Negative    114
Neutral      51
Name: count, dtype: int64
Sample labels: ['Negative', 'Negative', 'Negative', 'Positive', 'Neutral']


In [28]:
# final check: show shape, columns, and a quick preview
print(f"Final cleaned shape: {df_us.shape}")
print("Columns:", df_us.columns.tolist())

print("\nTop 10 categories by count:")
print(df_us["category"].value_counts().head(10))

print("\nSentiment label counts:")
print(df_us["sentiment_label"].value_counts())

print("\nSentiment score summary:")
print(df_us["sentiment_score"].describe())

# show first few rows of cleaned data
df_us.head(10)


Final cleaned shape: (291, 18)
Columns: ['post_id', 'platform', 'bank_name', 'post_text', 'category', 'subcategory', 'sentiment_label', 'sentiment_score', 'date', 'user_id', 'user_followers', 'likes', 'shares', 'replies', 'language', 'is_verified', 'resolved_flag', 'source_url']

Top 10 categories by count:
category
Atm Service          30
Branch Experience    26
Online Banking       25
Fees & Charges       24
Interest Rates       23
Customer Service     23
Account Opening      21
Credit Card          19
Loan Process         17
Notifications        17
Name: count, dtype: int64

Sentiment label counts:
sentiment_label
Positive    126
Negative    114
Neutral      51
Name: count, dtype: int64

Sentiment score summary:
count    291.000000
mean       0.058330
std        0.613851
min       -1.000000
25%       -0.424500
50%        0.034000
75%        0.558500
max        1.000000
Name: sentiment_score, dtype: float64


,post_id,platform,bank_name,post_text,category,subcategory,sentiment_label,sentiment_score,date,user_id,user_followers,likes,shares,replies,language,is_verified,resolved_flag,source_url
11,P000012,Yelp,GlobalBank,Cash shortage at ATM during the weekend ruined...,Atm Service,Cash Shortage,Negative,-1.000,2024-04-07,U25860,19,2,1,2,en,False,NaN,https://example.com/post/12
16,P000017,Forums,Horizon Bank,Why is the monthly fee increasing? Feels like ...,Fees & Charges,Monthly Fees,Negative,-0.756,2024-07-09,U17989,9,8,0,2,en,True,NaN,https://example.com/post/17
17,P000018,Reddit,Pioneer Credit,The banking app keeps crashing when I try to t...,App Performance,Login Issues,Negative,-1.000,2024-08-06,U42271,16,2,3,2,en,False,NaN,https://example.com/post/18
29,P000030,Google Play,Maple Savings,Session keeps timing out while I complete form...,Online Banking,Feature Request,Positive,0.449,2024-11-10,U92960,20,4,1,4,en,False,NaN,https://example.com/post/30
46,P000047,Reddit,Maple Savings,Loan officer was professional and the approval...,Loan Process,Mortgage,Neutral,0.133,2024-06-30,U26591,81,5,0,1,en,False,NaN,https://example.com/post/47
47,P000048,Twitter,GlobalBank,Long wait time at the branch and only one tell...,Branch Experience,Staff Behaviour,Negative,-0.967,2023-09-16,U72616,8,3,2,4,en,False,False,https://example.com/post/48
52,P000053,Twitter,Citadel Bank,Received phishing email pretending to be the b...,Security & Fraud,Unauthorized Transaction,Neutral,-0.062,2024-11-21,U85392,51,7,1,3,en,False,NaN,https://example.com/post/53
62,P000063,Yelp,SafeTrust Bank,Cash shortage at ATM during the weekend ruined...,Atm Service,Network Down,Positive,0.391,2023-11-03,U13794,9,2,0,1,en,False,NaN,https://example.com/post/63
70,P000071,Google Play,First National Bank,Exchange rate was competitive and transfer arr...,International Transfers,High Fees,Negative,-0.324,2024-04-20,U49608,7,6,0,1,en,False,True,https://example.com/post/71
74,P000075,LinkedIn,Pioneer Credit,Savings interest rates are abysmal compared to...,Interest Rates,Promotional Rate,Negative,-1.000,2024-07-07,U42853,19,5,0,0,en,False,NaN,https://example.com/post/75
